## References

- [DDSファイルを自力で読んでみよう](https://techblog.sega.jp/entry/2016/12/26/100000)
- [DirectX 11の圧縮フォーマットBC1～BC7について（前編）](https://www.webtech.co.jp/blog/optpix_labs/format/6993/)

- [DDSフォーマット](https://dench.flatlib.jp/ddsformat)

- [Direct3D10のマニュアル](https://learn.microsoft.com/en-us/windows/win32/direct3d10/d3d10-graphics-programming-guide-resources-block-compression#bc4)
  - 多分これが一番正確

- [Switch Toolboxの実装](https://github.com/KillzXGaming/Switch-Toolbox/blob/5b80a3d0f8ccadc4cea494c6037b07b4fa3032c5/Switch_Toolbox_Library/FileFormats/DDSCompressor.cs#L185)

In [ ]:
%cd ../

In [ ]:
import os
import struct
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
class DDS:
    HEADER_SIZE = 128
    WIDTH_OFFSET = 0x10
    HEIGHT_OFFSET = 0x0C
    FORMAT_OFFSET = 0x54

    HEADER_STRUCT = {
        'dwMagic': 'cccc',          # always 0x20534444 (' SDD')
        'dwSize': 'I',              # always 124
        'dwFlags': 'I',             # ヘッダ内の有効な情報 DDSD_* の組み合わせ
        'dwHeight': 'I',            # 画像の高さ x size
        'dwWidth': 'I',             # 画像の幅   y size
        'dwPitchOrLinearSize': 'I', # 横1 line の byte 数 (pitch) または 1面分の byte 数 (linearsize)
        'dwDepth': 'I',             # 画像の奥行き z size (Volume Texture 用)
        'dwMipMapCount': 'I',       # 含まれている mipmap レベル数
        'dwReserved1': 'I'*11,
        'dwPfSize': 'I',            # 常に 32
        'dwPfFlags': 'cccc',        # pixel フォーマットを表す DDPF_* の組み合わせ
        'dwFourCC': 'cccc',         # フォーマットが FourCC で表現される場合に使用する。 # DX10 拡張ヘッダが存在する場合は 'DX10' (0x30315844) が入る。
        'dwRGBBitCount': 'I',       # 1 pixel の bit 数
        'dwRBitMask': 'I',          # RGB format 時の mask
        'dwGBitMask': 'I',          # RGB format 時の mask
        'dwBBitMask': 'I',          # RGB format 時の mask
        'dwRGBAlphaBitMask': 'I',   # RGB format 時の mask
        'dwCaps': 'I',              # mipmap 等のフラグ指定用
        'dwCaps2': 'I',             # cube/volume texture 等のフラグ指定用
        'dwReservedCaps2': 'I'*2,
        'dwReserved2': 'I'
    }

    def __init__(self, filepath: str) -> None:
        self.filepath = Path(filepath)
        if not self.filepath.exists():
            raise FileNotFoundError(f'{filepath} is not exists, or not file')
        if self.filepath.is_dir():
            raise IsADirectoryError(f'{filepath} is a directory.')

        self.data: bytes = self.filepath.read_bytes()

        dds_header = struct.Struct("<"+''.join(self.HEADER_STRUCT.values()))
        opts = dds_header.unpack_from(self.data, 0)
        self.opt = {}
        index = 0
        for key, format in self.HEADER_STRUCT.items():
            res = []
            for c in format:
                if c == 'c':
                    res.append(chr(int.from_bytes(opts[index])))
                else:
                    res.append(opts[index])
                index += 1
            if len(res) == 1:
                self.opt[key] = res[0]
            elif format in {'cccc', '4c'}:
                self.opt[key] = ''.join(res)
            else:
                self.opt[key] = res

        self._start_index = self.HEADER_SIZE + (20 if self.opt['dwFourCC'] == 'DX10' else 0)

    def load_chunks(self) -> bytes:
        pass

    @property
    def header(self) -> bytes:
        return self.data[:self.HEADER_SIZE]

    @property
    def typeFormat(self) -> str:
        btype = self.data[self.FORMAT_OFFSET:self.FORMAT_OFFSET+4]
        return ''.join([chr(b) for b in btype])

    @property
    def width(self) -> int:
        return int.from_bytes(self.data[self.WIDTH_OFFSET:self.WIDTH_OFFSET+4], byteorder='little')

    @property
    def height(self) -> int:
        return int.from_bytes(self.data[self.HEIGHT_OFFSET:self.HEIGHT_OFFSET+4], byteorder='little')


In [ ]:
dds_filepath = "data/Sheet_0.dds"
dds = DDS(filepath=dds_filepath)

In [ ]:
dds.opt

In [ ]:
def view(data: bytes, start_offset: int = 0, upper: bool = True, show_ascii: bool = False) -> None:
    """print a bytes object like binary-viewer format"""
    if type(data) is not bytes:
        print("[ERROR] argument 'data' is not bytes object.")
        return
    data = data[start_offset:]
    n_row = len(data) // 0x0F
    print(n_row, "rows")
    print('ADDRESS  : 00 01 02 03 04 05 06 07 08 09 0A 0B 0C 0D 0E 0F')
    print('-' * (58 + (19 if show_ascii else 0)))
    for i in range(n_row):
        start = i * 0x10
        print(f'{start:08x} : ', end="")
        d = data[start:start+0x10]
        hexs = [f'{t:02x}' for t in d]
        if upper:
            hexs = [b.upper() for b in hexs]
            print(*hexs, sep=' ', end='')
        if show_ascii:
            characters = [chr(t) for t in d]
            characters = [(c if 20<=ord(c)<127 else '.') for c in characters]
            print(" "*3, end="")
            print(*characters, sep="")
        else:
            print()

In [ ]:
view(dds.header, show_ascii=True)

In [ ]:
def decode_bc4u_block(block: bytes, verbose: bool = False, return_numpy: bool = True):
    if len(block) != 8:
        raise ValueError(f'block length is required 8, not {len(block)}.')
    p0 = block[0]
    p1 = block[1]
    if verbose:
        print("代表値0:", p0)
        print("代表値1:", p1)
    # dt = block[2:]
    # Little Endianを考慮しつつ3x16 bitsを抽出
    # ここが違う。メモを見てリトルエンディアンを生成する。
    # 49 92 24 49 92 24 は綺麗に [001 001, ...]になる。要確認
    dt =  block[6:8][::-1] + block[2:6][::-1]

    # データ部分をビット文字列で格納
    raw = ''.join([bin(t)[2:].zfill(8) for t in dt])
    if verbose:
        print("raw:", ''.join([c+(' ' if i % 3 == 0 else '') for i, c in enumerate(raw, 1)]))

    pixels_bits = [int(raw[i:i+3], base=2) for i in range(0, len(raw), 3)]

    # Ref: https://www.webtech.co.jp/blog/optpix_labs/format/4569/
    # BC4 UNORM interpolation
    colors = {}
    colors[0] = p0
    colors[1] = p1
    if p0 > p1:
        # 6 interpolated color values
        for i in range(2, 8):
            colors[i] = (p0*(7-i+1) + p1*(i-1)) / 7
    else:
        # 4 interpolated color values
        for i in range(2, 6):
            colors[i] = (p0*(5-i+1) + p1*(i-1)) / 5
        colors[6] = 0
        colors[7] = 255

    pixels = [colors[int(b)] for b in pixels_bits]

    # pixels = [
    #     pixels[:8][::-1] + pixels[8:][::-1]
    # ]
    pixels = pixels[8:][::-1] + pixels[:8][::-1]

    # 予想だが、p0==1 and p1==0 の場合は処理をスキップして全部透明にしている気がする。
    # if p0 == 1 and p1 == 0:
    #     pixels = [0] * 16

    # if p0 > p1:
    #     pixels = [0] * 16
    # これも予想だが、特定のパターン 010 010 011 001 001 000 100 100 010 010 011 001 001 000 100 100 の場合は白塗りにしている気がする。
    # if raw == '010010011001001000100100010010011001001000100100':
    #     pixels = [0] * 16


    if verbose:
        print("pixels_bits:", pixels_bits)
        print(pixels)
    if return_numpy:
        return np.array(pixels).reshape(4, 4).astype(np.uint8)
    else:
        return pixels


In [ ]:
# ind = 12390
ind = 0
decode_bc4u_block(dds.data[dds._start_index+8*ind: dds._start_index+8*ind+8], verbose=True)

In [ ]:
orig = [6, 1, 4, 6, 7, 7, 7, 4, 7, 5, 4, 6, 7, 7, 4, 7]
print(orig)
print(orig[8:] + orig[:8])

In [ ]:
# decode all 4x4 block sequencially
blocks = []
for i in range(0, dds.width * dds.height // 2, 8):
    start_index = dds._start_index + i
    block = decode_bc4u_block(dds.data[start_index:start_index+8], return_numpy=True)
    blocks.append(block)

len(blocks)

In [ ]:
s = np.array(blocks)
n_blocks = dds.width // 4
hs = []
for n in range(n_blocks):
    hs.append(np.hstack(s[n*256:n*256+256]))
img = np.vstack(hs)

In [ ]:
img

In [ ]:
# img = np.random.random(16).reshape(4, 4)
# white = np.ones(dds.width * dds.height).reshape(dds.width, dds.height).astype(np.float32)
# white = cv2.cvtColor(white, cv2.COLOR_GRAY2RGBA)
# white[:, :, -1] = img.astype(np.float32) / 255.
# white = img.astype(np.float32)
white = img

fig, ax = plt.subplots(figsize=(20, 20), dpi=200)
ax.imshow(white, cmap='gray')


In [ ]:
cv2.imwrite('images/sample6.png', img.astype(np.uint8))